<div align="center">

# Градиентный бустинг: обучение ансамбля на основе градиентов потерь

</div>

---

## **Градиентный бустинг (Gradient Boosting)**

### Определение

* Вариант бустинга: **последовательное обучение слабых учеников** для создания сильного ансамбля.


### Важность

* Один из **ключевых методов** в современном ML.
* Широко применяется в соревнованиях (например, **Kaggle**) и реальных проектах.

---

## **Сравнение AdaBoost и Gradient Boosting**

| Характеристика           | **AdaBoost**                                                                                                  | **Gradient Boosting**                                                                                       |
| ------------------------ | ------------------------------------------------------------------------------------------------------------- | ----------------------------------------------------------------------------------------------------------- |
| **Базовые модели**       | Обычно **обрубки деревьев** (decision stumps, глубина = 1)                                                    | Деревья глубже (глубина ≈ 3–6 или 8–64 листьев)                                                             |
| **Подход к обучению**    | Итеративно обучает классификаторы на **взвешенных данных** — ошибки предыдущей модели влияют на веса примеров | Итеративно обучает деревья на **остатках ошибок** (градиентах) — ошибки становятся новой целевой переменной |
| **Веса**                 | Индивидуальные веса для каждого примера и каждого классификатора                                              | **Глобальная скорость обучения** (learning rate), одинаковая для всех деревьев                              |
| **Критерий остановки**   | Достижение максимального числа итераций                                                                       | Достижение максимального числа деревьев или другой критерий                                                 |
| **Интерпретация ошибок** | Используются напрямую для перераспределения весов                                                             | Преобразуются в новые "целевые" значения для следующей модели                                               |

### Главное отличие в механизме

* **AdaBoost**: меняет **веса примеров** → слабые ученики фокусируются на сложных объектах.
* **Gradient Boosting**: строит новые модели, минимизируя **градиент функции потерь** → ошибки становятся новым таргетом.


---

## **Градиентный бустинг — общий алгоритм (пример: бинарная классификация)**

### Идея

* Строим последовательность деревьев решений.
* Каждое дерево обучается на **ошибках (остатках)** предыдущего ансамбля.
* Ошибки определяются как **отрицательный градиент функции потерь**.
* Каждое новое дерево делает **небольшой шаг** в направлении уменьшения ошибки (через *learning rate*).


### **Пошаговый алгоритм**

#### **Шаг 1 — Инициализация**

* Строим **одноузловое дерево** (корневой узел), выдающее константу $y_0$.
* Константа находится как:

$$
F_0(x) = \arg\min_{\gamma} \sum_{i=1}^n L(y_i, \gamma)
$$

где:

* $L$ — функция потерь,
* $n$ — число примеров.


#### **Шаг 2 — Итерации (для $m = 1, \dots, M$)**

**a. Вычисляем псевдоостатки (pseudo-residuals)**:

$$
r_{im} = - \left[ \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right]_{F(x)=F_{m-1}(x)}
$$

**b. Строим дерево решений** по $r_{im}$ как целевой переменной.
Пусть дерево имеет $J_m$ листьев с областями $R_{jm}$.

**c. Для каждого листа $R_{jm}$**:

$$
\gamma_{jm} = \arg\min_{\gamma} \sum_{x_i \in R_{jm}} L\big(y_i, F_{m-1}(x_i) + \gamma\big)
$$

— вычисляем оптимальное смещение $\gamma_{jm}$, минимизирующее функцию потерь.

**d. Обновляем модель**:

$$
F_m(x) = F_{m-1}(x) + \eta \cdot \sum_{j=1}^{J_m} \gamma_{jm} \cdot \mathbf{1}(x \in R_{jm})
$$

где:

* $\eta$ — **скорость обучения** (*learning rate*, 0.01–1),
* $\mathbf{1}(\cdot)$ — индикатор принадлежности объекта листу.

### Особенности

* **Псевдоостатки** = направление, в котором нужно улучшить прогноз (градиент потерь).
* **Learning rate ($\eta$)**: замедляет обновление, снижает риск переобучения.
* **Глубина деревьев**: обычно 3–6, но можно настраивать.

---

## **Практические реализации градиентного бустинга**

### Проблема классической версии

* Градиентный бустинг — **последовательный процесс**, обучение может быть **медленным**.


### **XGBoost (eXtreme Gradient Boosting)**

* **Оптимизации**:

  * Приёмы и приближения для ускорения обучения.
  * Эффективное использование памяти.
  * Регуляризация для борьбы с переобучением.
* **Сильные стороны**:

  * Высокая скорость обучения.
  * Отличные прогностические характеристики.
* **Популярность**:

  * Победы во многих соревнованиях Kaggle.

### Другие реализации

* **LightGBM** — от Microsoft, ориентирован на скорость и низкое потребление памяти.
* **CatBoost** — от Яндекса, лучше работает с категориальными признаками "из коробки".
* **HistGradientBoostingClassifier** (scikit-learn) — основан на подходах LightGBM, быстрее классического `GradientBoostingClassifier`.

In [5]:
# XGBoost соответсвует API scikit-learn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
# Загрузка данных
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/'
                      'wine/wine.data',
                      header = None)
df_wine.columns = ['Class label', 'Alcohol',
                   'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium',
                   'Total phenols', 'Flavanoids',
                   'Nonflavanoid phenols',
                   'Proanthocyanins',
                   'Color intensity', 'Hue',
                   'OD280/OD315 of diluted wines',
                   'Proline']

# Отбрасываем класс 1
df_wine = df_wine[df_wine['Class label'] != 1]

# Разделение данных
X = df_wine[['Alcohol', 'OD280/OD315 of diluted wines']].values
y = df_wine['Class label'].values

df_wine[['Class label', 'Alcohol', 'OD280/OD315 of diluted wines']].head()

# Кодирование меток класса в двоичный формат
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size = 0.2,
                     random_state = 1,
                     stratify = y)

In [8]:
model = xgb.XGBClassifier(n_estimators = 1000, 
                          learning_rate = 0.01,
                          max_depth = 4,
                          random_state = 1)

gbm = model.fit(X_train, y_train)
y_train_pred = gbm.predict(X_train)
y_test_pred = gbm.predict(X_test)
gbm_train = accuracy_score(y_train, y_train_pred)
gbm_test = accuracy_score(y_test, y_test_pred)
print(f'Точность XGBoost при обучении/тестировании '
      f'{gbm_train:.3f} / {gbm_test:.3f}')

Точность XGBoost при обучении/тестировании 0.968 / 0.917


Мы обучили классификатор на основе градиентного бустинга с `1000 деревьев (раундов)` и `скоростью обучения = 0.001`. Посколько мы поощряем слабых учеников, значения от 2 до 6 является разумнынм выбором.

---


## **Вывод по ансамблевым методам**

### **Общая идея ансамблей**

* Ансамблевые методы объединяют **несколько моделей**, чтобы компенсировать их слабые стороны.
* Результат — **стабильные и точные модели**, полезные как для **производственных задач**, так и для **соревнований по машинному обучению**.


### **Мажоритарное голосование (Majority Voting)**

* Простейший способ объединения предсказаний нескольких моделей.
* Каждая модель делает прогноз, а **итоговое решение выбирается по большинству голосов**.
* Используется как базовый метод для ансамблей и для **бэггинга**.


### **Бэггинг (Bagging)**

* Уменьшает **дисперсию модели** за счет:

  * Создания **случайных бутстрэп-выборок** из обучающего набора.
  * Обучения **нескольких независимых моделей**.
  * Объединения их **по принципу мажоритарного голосования**.
* Основной эффект: **устойчивость модели к шуму** и переобучению.


### **Бустинг**

* Использует **последовательное обучение слабых учеников**, которые учатся на ошибках предыдущих моделей.
* Основные виды:

  * **AdaBoost**:

    * Взвешивает ошибки выборки для каждой итерации.
    * Слабые модели (обычно обрубки деревьев решений) комбинируются в сильный классификатор.
  * **Градиентный бустинг**:

    * Постепенно корректирует ошибки предыдущих деревьев через **градиент функции потерь**.
    * Популярные реализации:

      * **XGBoost** — ускоренная и регуляризованная версия градиентного бустинга.
      * **LightGBM** — оптимизация скорости и памяти.
      * **CatBoost** — удобен для категориальных признаков.
      * **HistGradientBoostingClassifier** (scikit-learn) — эффективная версия градиентного бустинга.


**Итог**:

* Мажоритарное голосование → простое объединение моделей.
* Бэггинг → уменьшение дисперсии через независимые выборки и голосование.
* Бустинг → последовательное обучение на ошибках для повышения точности.
* Ансамбли повышают **качество прогнозов**, но могут увеличивать **вычислительные затраты**.